# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [63]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [64]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [65]:
%%capture --no-display

# Configure the map plot
humiditygraph = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 500,
    frame_width = 896,
    hover_cols =["City", "Humidity"]
)

# Display the map
humiditygraph

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [66]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     577 non-null    int64  
 1   City        577 non-null    object 
 2   Lat         577 non-null    float64
 3   Lng         577 non-null    float64
 4   Max Temp    577 non-null    float64
 5   Humidity    577 non-null    int64  
 6   Cloudiness  577 non-null    int64  
 7   Wind Speed  577 non-null    float64
 8   Country     574 non-null    object 
 9   Date        577 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 45.2+ KB


In [67]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 0
max_temp = 40
max_wind = 10

# Drop any rows with null values
city_data_df.dropna()

mask = (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] < max_temp) & (city_data_df["Wind Speed"] < max_wind)
city_data_df = city_data_df.loc[mask].reset_index(drop=True)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [68]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     521 non-null    int64  
 1   City        521 non-null    object 
 2   Lat         521 non-null    float64
 3   Lng         521 non-null    float64
 4   Max Temp    521 non-null    float64
 5   Humidity    521 non-null    int64  
 6   Cloudiness  521 non-null    int64  
 7   Wind Speed  521 non-null    float64
 8   Country     518 non-null    object 
 9   Date        521 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 40.8+ KB


### Step 3: Create a new DataFrame called `hotel_df`.

In [70]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,None
1,farsund,NO,58.0948,6.8047,100,None
2,new norfolk,AU,-42.7826,147.0587,58,None
3,jamestown,US,42.0970,-79.2353,77,None
4,lanzhou,CN,36.0564,103.7922,48,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [71]:

# Set parameters to search for a hotel
radius = 10000  # search radius in meters (10 km)
categories = "accommodation.hotel"  # Geoapify category for hotels
limit = 5  # Return only the nearest 5 hotels
geoapify_key = "d15c9d0029bc45e7a1c5d9511f3ef1ff"  # Your Geoapify API key

# Print a message to follow up the hotel search
print("Starting hotel search")

# Check if the API key is correct
print(f"Using Geoapify API Key: {geoapify_key}")  # Debugging line, can be removed later

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Update the filters and bias with the correct lat/lng
    filters = f"circle:{lng},{lat},{radius}"  # Correct filter for search radius
    bias = f"proximity:{lng},{lat}"  # Correct bias for proximity

    # Set the params dictionary
    params = {
        "categories": categories,
        "limit": limit,
        "filter": filters,
        "bias": bias,
        "apiKey": geoapify_key
    }

    # Set base URL for Geoapify Places API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    try:
        response = requests.get(base_url, params=params)

        # Check if the response is successful (status code 200)
        if response.status_code != 200:
            print(f"Error: Received status code {response.status_code} for {row['City']}")
            hotel_df.loc[index, "Hotel Name"] = "Error in API request"
            continue
        
        # Convert the API response to JSON format
        name_address = response.json()

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        if "features" in name_address and len(name_address["features"]) > 0:
            hotel_name = name_address["features"][0]["properties"]["name"]
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    except (KeyError, IndexError, requests.exceptions.RequestException) as e:
        # Handle errors such as missing data or API request failure
        hotel_df.loc[index, "Hotel Name"] = "Error in API request"
        print(f"Error fetching hotel for {row['City']}: {e}")

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the DataFrame to check results
hotel_df.head()

Starting hotel search
Using Geoapify API Key: d15c9d0029bc45e7a1c5d9511f3ef1ff
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Woodbridge on the Derwent
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: Hotel SØMA
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
Error fetching hotel for nikolskoye: 'name'
nikolskoye - nearest hotel: Error in API request
trairi - nearest hotel: Pousada Villa Aurora
dobryanka - nearest hotel: Уральская Венеция
mataura - nearest hotel: No hotel found
tasiilaq - nearest hotel: Angmagssalik
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
nuevo laredo - nearest hotel: Best Western Plus
guiratinga - nearest hotel: No hote

,City,Country,Lat,Lng,Humidity,Hotel Name
0,faya,SA,18.3851,42.4509,35,No hotel found
1,farsund,NO,58.0948,6.8047,100,Rederiet Hotell
2,new norfolk,AU,-42.7826,147.0587,58,Woodbridge on the Derwent
3,jamestown,US,42.0970,-79.2353,77,DoubleTree Jamestown
4,lanzhou,CN,36.0564,103.7922,48,西园宾馆


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [72]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",          
    "Lat",          
    geo=True,       
    tiles="EsriNatGeo",  # Choose the tile layer
    frame_width=700,
    frame_height=500,
    color="City",       # You can customize the color based on the city name or any other variable
    hover_cols=["City", "Country", "Hotel Name"]  # Display these columns in the hover popup
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)